In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ai-vs-human-text/AI_Human.csv


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import nltk
from nltk.corpus import stopwords
import string

In [3]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df = pd.read_csv('/kaggle/input/ai-vs-human-text/AI_Human.csv')

In [5]:
df.head()

,text,generated
0,Cars. Cars have been around since they became ...,0.0
1,Transportation is a large necessity in most co...,0.0
2,"""America's love affair with it's vehicles seem...",0.0
3,How often do you ride in a car? Do you drive a...,0.0
4,Cars are a wonderful thing. They are perhaps o...,0.0


In [6]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

In [7]:
df['text'] = df['text'].apply(preprocess_text)

In [8]:
X = df['text']
y = df['generated']

In [9]:
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(X)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.3, random_state=42)

In [11]:
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [12]:
y_pred = model.predict(X_test)

In [13]:
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.9522955989902238
Classification Report:
              precision    recall  f1-score   support

         0.0       0.94      0.99      0.96     91597
         1.0       0.98      0.89      0.93     54574

    accuracy                           0.95    146171
   macro avg       0.96      0.94      0.95    146171
weighted avg       0.95      0.95      0.95    146171



In [14]:
def predict_text_category(input_text):
    processed_text = preprocess_text(input_text)
    transformed_text = vectorizer.transform([processed_text])
    prediction = model.predict(transformed_text)
    if prediction[0] == 1:
        return 'AI-generated'
    else:
        return 'Human-written'

def predict_text_category(input_text):
    processed_text = preprocess_text(input_text)
    transformed_text = vectorizer.transform([processed_text])
    prediction = model.predict(transformed_text)
    
    if prediction[0] == 0:
        return 'Human-written'
    elif prediction[0] == 1:
        return 'AI-generated (Model: GPT)'
    elif prediction[0] == 2:
        return 'AI-generated (Model: BERT)'
    # Add more elif cases for other models if needed
    else:
        return 'AI-generated (Model: Unknown)'

In [15]:
input_text = input("Enter a text: ")
result = predict_text_category(input_text)
print(f'The text is likely {result}.')

StdinNotImplementedError: raw_input was called, but this frontend does not support input requests.